# 01 — Generate + Score (pipeline)

Notebook de orquestração: chama os scripts oficiais do repo para **gerar** `submission.csv` e **scorar** (rápido e strict).

Tem 2 workflows:

1) **Run único**: `python -m santa_packing` (gera + melhora + valida + arquiva)
2) **Experimentos massivos**: `sweep_ensemble` (multi-start + *ensemble por n*) → gera tabelas e facilita tirar conclusões (qual recipe domina quais `n`).

Recomendação: antes de confiar em um resultado, use **Restart & Run All**.

Outputs ficam em `runs/notebook_runs/` (ignorado pelo git).


In [1]:
from __future__ import annotations

import csv
import json
import shlex
import subprocess
import sys
import time
from pathlib import Path


def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for cand in (start, *start.parents):
        if (cand / "pyproject.toml").is_file():
            return cand
    return start


ROOT = find_repo_root(Path.cwd())
OUT_DIR = ROOT / "runs" / "notebook_runs"
OUT_DIR.mkdir(parents=True, exist_ok=True)


def run(cmd: list[str], *, capture: bool = False) -> subprocess.CompletedProcess[str]:
    print("+", " ".join(shlex.quote(c) for c in cmd))
    return subprocess.run(
        cmd,
        cwd=str(ROOT),
        text=True,
        capture_output=capture,
        check=True,
    )


def git_sha() -> str:
    try:
        proc = subprocess.run(
            ["git", "rev-parse", "HEAD"],
            cwd=str(ROOT),
            text=True,
            capture_output=True,
            check=True,
        )
        return (proc.stdout or "").strip()
    except Exception:
        return "unknown"


def python_has_jax(python: str) -> bool:
    try:
        subprocess.run([python, "-c", "import jax"], check=True, capture_output=True, text=True)
        return True
    except Exception:
        return False


def default_python() -> str:
    # Prefer the current kernel, but fallback to the repo .venv if needed.
    exe = sys.executable
    if python_has_jax(exe):
        return exe
    cand = ROOT / ".venv" / "bin" / "python"
    if cand.is_file() and python_has_jax(str(cand)):
        print("[info] Using repo .venv python:", cand)
        return str(cand)
    return exe


In [2]:
# MODE:
# - "make_submit": usa `python -m santa_packing` (workflow único; gera + score strict + arquiva)
# - "manual": chama `python -m santa_packing.cli.generate_submission` + `score_submission`
MODE = "make_submit"

# Comuns
PYTHON = default_python()
NMAX = 200
NAME = "notebook"
SEED = 1  # None para nao passar --seed
OVERLAP_MODE = "strict"  # strict / conservative / kaggle

# make_submit
CONFIG = "configs/submit.json"  # opcional; use "" para nao passar --config
GEN_EXTRA_ARGS = ""  # ex.: "--refine-steps 2000 --refine-batch 64 --refine-proposal mixed --refine-neighborhood"

# manual
GEN_ARGS = ""  # ex.: "--refine-steps 2000 --refine-batch 64 --refine-proposal mixed --refine-neighborhood"


In [ ]:
ts = time.strftime("%Y%m%d-%H%M%S")
sha = git_sha()

run_dir: Path | None = None
submission_path: Path | None = None
strict_score: dict

if MODE == "make_submit":
    cmd = [
        PYTHON,
        "-m",
        "santa_packing",
        "--nmax",
        str(int(NMAX)),
        "--name",
        NAME,
        "--submissions-dir",
        str(OUT_DIR),
    ]
    cmd += ["--overlap-mode", str(OVERLAP_MODE)]
    if SEED is not None:
        cmd += ["--seed", str(int(SEED))]
    if CONFIG.strip():
        cmd += ["--config", CONFIG]
    cmd += ["--"] + shlex.split(GEN_EXTRA_ARGS)

    proc = run(cmd, capture=True)
    print(proc.stdout)

    submission_path = None
    for line in (proc.stdout or "").splitlines():
        if line.startswith("Run:"):
            run_dir = Path(line.split(":", 1)[1].strip())
        if line.startswith("Submission:"):
            submission_path = Path(line.split(":", 1)[1].strip())
    if run_dir is None:
        raise RuntimeError("Não consegui extrair o run_dir do stdout do workflow")
    if submission_path is None:
        submission_path = run_dir / "submission_final.csv"
    strict_score = json.loads((run_dir / "score.json").read_text(encoding="utf-8"))
else:
    submission_path = OUT_DIR / f"submission_{ts}.csv"
    cmd_gen = [
        PYTHON,
        "-m",
        "santa_packing.cli.generate_submission",
        "--out",
        str(submission_path),
        "--nmax",
        str(int(NMAX)),
        "--overlap-mode",
        str(OVERLAP_MODE),
    ]
    if SEED is not None:
        cmd_gen += ["--seed", str(int(SEED))]
    cmd_gen += shlex.split(GEN_ARGS)
    run(cmd_gen)

    cmd_strict = [
        PYTHON,
        "-m",
        "santa_packing.cli.score_submission",
        str(submission_path),
        "--nmax",
        str(int(NMAX)),
        "--overlap-mode",
        str(OVERLAP_MODE),
        "--pretty",
    ]
    proc_strict = run(cmd_strict, capture=True)
    strict_score = json.loads(proc_strict.stdout or "{}")

print("Strict score:", strict_score.get("score"))


+ /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/.venv/bin/python -m santa_packing --nmax 200 --name notebook --submissions-dir /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/runs/notebook_runs --overlap-mode strict --seed 1 -- --sa-nmax 30 --sa-steps 400 --sa-batch 64 --sa-proposal mixed --sa-neighborhood --sa-objective packing


In [ ]:
cmd_fast = [
    PYTHON,
    "-m",
    "santa_packing.cli.score_submission",
    str(submission_path),
    "--nmax",
    str(int(NMAX)),
    "--no-overlap",
]
proc_fast = run(cmd_fast, capture=True)
fast_score = json.loads(proc_fast.stdout or "{}")
print("Fast score:", fast_score.get("score"))

if run_dir is not None:
    (run_dir / "score_fast.json").write_text(json.dumps(fast_score, indent=2) + "\n", encoding="utf-8")


In [ ]:
experiments_csv = OUT_DIR / "experiments.csv"

row = {
    "timestamp": ts,
    "git_sha": sha,
    "mode": MODE,
    "name": NAME,
    "nmax": int(NMAX),
    "seed": "" if SEED is None else int(SEED),
    "overlap_mode": str(OVERLAP_MODE),
    "config": CONFIG if MODE == "make_submit" else "",
    "gen_args": GEN_ARGS if MODE != "make_submit" else GEN_EXTRA_ARGS,
    "run_dir": str(run_dir) if run_dir is not None else "",
    "submission": str(submission_path),
    "score_strict": strict_score.get("score"),
    "s_max_strict": strict_score.get("s_max"),
    "score_fast": fast_score.get("score"),
    "s_max_fast": fast_score.get("s_max"),
}

fields = list(row.keys())
is_new = not experiments_csv.exists()

with experiments_csv.open("a", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=fields)
    if is_new:
        w.writeheader()
    w.writerow(row)

print("Logged:", experiments_csv)

# Show last 10 rows (no pandas)
rows = []
with experiments_csv.open("r", newline="", encoding="utf-8") as f:
    r = csv.DictReader(f)
    rows = list(r)

for r in rows[-10:]:
    print(r["timestamp"], r["mode"], r["score_strict"], r["submission"])


## Melhorar um submission (improve_submission)

Quando o baseline trava com score alto (ex.: 150+), um ganho grande e rápido costuma vir de:

1) **subset-smoothing**: reutiliza soluções de `n` maiores para melhorar `n` menores (reduz “spikes” no score).
2) **melhora do `n=200`**: insere 1 árvore a partir do `n=199`, roda SA com vizinhança e depois finaliza/valida sem overlap.

A célula abaixo faz um sweep simples (janelas × seeds) e escreve uma tabelinha com os resultados.


In [ ]:
# --- Improve existing submission via improve_submission (massive but controllable).
import shutil

IMPROVE_RUN = True
IMPROVE_WRITE_TO_ROOT = True  # copia o melhor para ROOT/submission.csv (gitignored)

IMPROVE_DIR = OUT_DIR / "improve"
IMPROVE_DIR.mkdir(parents=True, exist_ok=True)

BASE = submission_path  # gerado na primeira parte do notebook
IMPROVE_NMAX = int(NMAX)

# Aumente esses ranges para rodar “massivo”.
IMPROVE_SMOOTH_WINDOWS = [20, 40, 60]
IMPROVE_SEEDS = [11, 22, 33, 44, 55]

IMPROVE_SA_STEPS = 6000
IMPROVE_SA_BATCH = 32


def score_strict_csv(path: Path) -> dict:
    cmd = [
        PYTHON,
        "-m",
        "santa_packing.cli.score_submission",
        str(path),
        "--nmax",
        str(int(IMPROVE_NMAX)),
        "--overlap-mode",
        str(OVERLAP_MODE),
        "--pretty",
    ]
    proc = run(cmd, capture=True)
    return json.loads(proc.stdout or "{}")


results: list[dict[str, object]] = []
best: dict[str, object] | None = None

if IMPROVE_RUN:
    for w in IMPROVE_SMOOTH_WINDOWS:
        for seed in IMPROVE_SEEDS:
            tag = f"w{int(w)}_seed{int(seed)}"
            out_csv = IMPROVE_DIR / f"submission_improved_{tag}.csv"
            cmd = [
                PYTHON,
                "-m",
                "santa_packing.cli.improve_submission",
                str(BASE),
                "--out",
                str(out_csv),
                "--nmax",
                str(int(IMPROVE_NMAX)),
                "--smooth-window",
                str(int(w)),
                "--overlap-mode",
                str(OVERLAP_MODE),
                "--improve-n200",
                "--n200-insert-seed",
                str(int(seed)),
                "--n200-sa-seed",
                str(int(seed)),
                "--n200-sa-steps",
                str(int(IMPROVE_SA_STEPS)),
                "--n200-sa-batch",
                str(int(IMPROVE_SA_BATCH)),
            ]
            run(cmd)
            strict = score_strict_csv(out_csv)
            s = float(strict.get("score") or float("inf"))
            row = {"tag": tag, "score": s, "s_max": strict.get("s_max"), "csv": str(out_csv)}
            results.append(row)
            if best is None or s < float(best["score"]):
                best = row
            print("improve:", tag, "score=", s)

print("\nBest improve:", best)

improve_csv = OUT_DIR / "improve_experiments.csv"
if results:
    fields = ["tag", "score", "s_max", "csv"]
    with improve_csv.open("w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fields)
        w.writeheader()
        w.writerows(results)
    print("Wrote:", improve_csv)

if IMPROVE_WRITE_TO_ROOT and best is not None:
    src = Path(str(best["csv"]))
    dst = ROOT / "submission.csv"
    shutil.copyfile(src, dst)
    print("Updated:", dst)


## Experimentos massivos (sweep_ensemble)

Para ir pra leaderboard, o caminho mais forte aqui é **multi-start + ensemble por n**.

O comando `python -m santa_packing.cli.sweep_ensemble`:

* roda vários `generate_submission` (varia seeds/recipes)
* calcula `s_n` de cada candidato
* monta um submission final escolhendo, **para cada n**, o candidato com menor `s_n` (com checagem de overlap opcional)

A seção abaixo roda sweeps grandes e depois analisa automaticamente **quais recipes dominam quais ranges de n**.


In [ ]:
# --- Massive sweep settings (pode demorar; essa e a ideia).
import os

# Evita recompilar JAX em cada subprocesso (sweep_ensemble roda varios processos).
JAX_CACHE_DIR = OUT_DIR / "jax_compilation_cache"
JAX_CACHE_DIR.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("JAX_COMPILATION_CACHE_DIR", str(JAX_CACHE_DIR))
print("JAX_COMPILATION_CACHE_DIR=", os.environ.get("JAX_COMPILATION_CACHE_DIR"))

SWEEP_NMAX = 200
SWEEP_SEEDS = "1..20"  # aceita "1,2,3" ou range "1..50"
SWEEP_JOBS = 1  # aumente para rodar candidatos em paralelo
SWEEP_TIMEOUT_S = None  # ex.: 3600 (1h) por candidato
SWEEP_REUSE = True  # reaproveita candidates/ se ja existirem
SWEEP_KEEP_GOING = True  # ignora candidatos falhos
SWEEP_OVERLAP_CHECK = "selected"  # "selected" (seguro) ou "none" (inseguro)

SWEEP_RUNS_DIR = OUT_DIR / "sweep_runs"
SWEEP_RUNS_DIR.mkdir(parents=True, exist_ok=True)

SWEEP_SESSION = time.strftime("%Y%m%d-%H%M%S") + "_" + git_sha()[:7]

# Portfolio style:
# - "baseline": parte de uma submission existente (step_0000) + opcional refine-SA
# - "mother"   : otimiza N=200 uma vez e emite prefixes (bem mais rapido pra sweeps massivos)
# - "mixed"    : usa o portfolio do repo (mais variado, mas tende a ser MUITO mais lento)
PORTFOLIO_STYLE = "baseline"

# Baseline (submission existente) - bom ponto de partida para chegar <70.
BASELINE_SUBMISSION = (
    ROOT / "runs" / "hunt70_from_submission" / "round_000" / "ensemble_safe_logs" / "_tmp_ensemble" / "step_0000.csv"
)

# Baseline refine (tune aqui)
BASELINE_REFINE_STEPS = 4000
BASELINE_REFINE_BATCH = 64

# Portfolio base (recipes) do repo
PORTFOLIO_BASE = ROOT / "scripts" / "submission" / "portfolios" / "mixed.json"

# A/B test: portfolio com vizinhanca vs sem vizinhanca
RUN_AB_NEIGHBORHOOD = True

# Dica: SA por-n (sa-nmax>0 sem mother-prefix) tende a ser MUITO mais lento por causa de compilacao por tamanho.
INCLUDE_SA_SMALL_RECIPES = False

# Recomendo deixar True e focar em mother-prefix + refine (otimiza N=200 uma vez e emite prefixes).
INCLUDE_MOTHER_RECIPES = True

# Mother recipes (tune aqui)
MOTHER_LATTICE_POST_STEPS = 30
MOTHER_REFINE_STEPS = 2000
MOTHER_REFINE_BATCH = 64

# Overwrites opcionais (se None, deixa o preset do --*-neighborhood)
REFINE_SWAP_PROB = None
REFINE_COMPACT_PROB = None
REFINE_TELEPORT_PROB = None


In [ ]:
def _python_has_jax(python: str) -> bool:
    try:
        subprocess.run([python, "-c", "import jax"], check=True, capture_output=True, text=True)
        return True
    except Exception:
        return False


if not _python_has_jax(PYTHON):
    raise RuntimeError(
        f"PYTHON={PYTHON} nao tem JAX; selecione o kernel .venv (ou ajuste PYTHON) para rodar SA/vizinhanca."
    )


def _load_portfolio(path: Path) -> list[dict]:
    data = json.loads(path.read_text(encoding="utf-8"))
    if isinstance(data, dict):
        data = data.get("recipes", [])
    if not isinstance(data, list):
        raise TypeError(f"Portfolio invalido: {path}")
    return [dict(x) for x in data]


def _write_portfolio(path: Path, recipes: list[dict]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps({"recipes": recipes}, indent=2) + "\n", encoding="utf-8")


def _strip_flags(args: str, flags: set[str]) -> str:
    toks = shlex.split(args)
    toks = [t for t in toks if t not in flags]
    return " ".join(toks)


base_recipes: list[dict] = []

if PORTFOLIO_STYLE == "mixed":
    base_recipes = _load_portfolio(PORTFOLIO_BASE)
    if not INCLUDE_SA_SMALL_RECIPES:
        base_recipes = [r for r in base_recipes if not str(r.get("name", "")).startswith("sa_")]
elif PORTFOLIO_STYLE == "baseline":
    if not BASELINE_SUBMISSION.is_file():
        raise FileNotFoundError(f"Baseline submission nao encontrada: {BASELINE_SUBMISSION}")
    refine_steps = int(BASELINE_REFINE_STEPS)
    refine_batch = int(BASELINE_REFINE_BATCH)
    base_recipes = [
        {
            "name": "baseline_step0000",
            "args": f"--from-submission {BASELINE_SUBMISSION}",
        },
        {
            "name": "baseline_refine_neigh",
            "args": f"--from-submission {BASELINE_SUBMISSION} --refine-steps {refine_steps} --refine-batch {refine_batch} --refine-proposal mixed --refine-neighborhood",
        },
    ]
else:
    # Mother-prefix baselines: solve N=200 once and emit prefixes (fast for sweeps).
    post_steps = int(MOTHER_LATTICE_POST_STEPS)
    post_args = "" if post_steps <= 0 else f" --lattice-post-nmax 200 --lattice-post-steps {post_steps}"

    base_recipes = [
        {
            "name": "mother_lattice_hex_checker",
            "args": f"--mother-prefix --mother-reorder radial --sa-nmax 0 --lattice-pattern hex --lattice-rotate-mode checker --lattice-rotations 0,15,30{post_args}",
        },
        {
            "name": "mother_lattice_square_checker",
            "args": f"--mother-prefix --mother-reorder radial --sa-nmax 0 --lattice-pattern square --lattice-rotate-mode checker --lattice-rotations 0,15,30{post_args}",
        },
    ]

if INCLUDE_MOTHER_RECIPES:
    refine_steps = int(MOTHER_REFINE_STEPS)
    refine_batch = int(MOTHER_REFINE_BATCH)

    def _maybe(flag: str, value) -> str:
        return f" {flag} {value}" if value is not None else ""

    refine_extra = (
        _maybe("--refine-swap-prob", REFINE_SWAP_PROB)
        + _maybe("--refine-compact-prob", REFINE_COMPACT_PROB)
        + _maybe("--refine-teleport-prob", REFINE_TELEPORT_PROB)
    )

    base_recipes = base_recipes + [
        {
            "name": "mother_refine_neigh_radial",
            "args": f"--mother-prefix --mother-reorder radial --sa-nmax 0 --refine-nmin 200 --refine-steps {refine_steps} --refine-batch {refine_batch} --refine-proposal mixed --refine-neighborhood --lattice-pattern hex --lattice-rotate-mode checker --lattice-rotations 0,15,30{refine_extra}",
        },
        {
            "name": "mother_refine_neigh_none",
            "args": f"--mother-prefix --mother-reorder none --sa-nmax 0 --refine-nmin 200 --refine-steps {refine_steps} --refine-batch {refine_batch} --refine-proposal mixed --refine-neighborhood --lattice-pattern hex --lattice-rotate-mode checker --lattice-rotations 0,15,30{refine_extra}",
        },
    ]
portfolio_neigh = OUT_DIR / f"portfolio_{SWEEP_SESSION}_neigh.json"
_write_portfolio(portfolio_neigh, base_recipes)

portfolios: list[tuple[str, Path]] = [("neigh", portfolio_neigh)]

if RUN_AB_NEIGHBORHOOD:
    flags = {"--sa-neighborhood", "--refine-neighborhood", "--neighborhood"}
    no_neigh = []
    for r in base_recipes:
        args = str(r.get("args", ""))
        r2 = dict(r)
        r2["args"] = _strip_flags(args, flags)
        no_neigh.append(r2)
    portfolio_no_neigh = OUT_DIR / f"portfolio_{SWEEP_SESSION}_no_neigh.json"
    _write_portfolio(portfolio_no_neigh, no_neigh)
    portfolios.append(("no_neigh", portfolio_no_neigh))

print("Portfolios:")
for label, path in portfolios:
    print("-", label, path)


In [8]:
def _safe_name(text: str) -> str:
    out = []
    for ch in text.strip():
        if ch.isalnum() or ch in {"-", "_"}:
            out.append(ch)
        else:
            out.append("_")
    cleaned = "".join(out).strip("_")
    return cleaned or "tag"


def _latest_run_dir(runs_dir: Path, tag: str) -> Path:
    prefix = _safe_name(tag) + "_"
    cands = [p for p in runs_dir.iterdir() if p.is_dir() and p.name.startswith(prefix)]
    if not cands:
        raise FileNotFoundError(f"Nenhum run_dir encontrado para tag={tag!r} em {runs_dir}")
    return max(cands, key=lambda p: p.stat().st_mtime)


def _score_submission_csv(csv_path: Path) -> dict:
    cmd = [
        PYTHON,
        "-m",
        "santa_packing.cli.score_submission",
        str(csv_path),
        "--nmax",
        str(int(SWEEP_NMAX)),
        "--overlap-mode",
        str(OVERLAP_MODE),
        "--pretty",
    ]
    proc = run(cmd, capture=True)
    return json.loads(proc.stdout or "{}")


def run_sweep(label: str, portfolio_json: Path) -> dict:
    tag = f"nb_{SWEEP_SESSION}_{label}"
    out_csv = OUT_DIR / f"ensemble_{_safe_name(tag)}.csv"
    cmd = [
        PYTHON,
        "-m",
        "santa_packing.cli.sweep_ensemble",
        "--runs-dir",
        str(SWEEP_RUNS_DIR),
        "--tag",
        tag,
        "--nmax",
        str(int(SWEEP_NMAX)),
        "--seeds",
        str(SWEEP_SEEDS),
        "--jobs",
        str(int(SWEEP_JOBS)),
        "--overlap-check",
        str(SWEEP_OVERLAP_CHECK),
        "--recipes-json",
        str(portfolio_json),
        "--out",
        str(out_csv),
    ]
    if SWEEP_TIMEOUT_S is not None:
        cmd += ["--timeout-s", str(float(SWEEP_TIMEOUT_S))]
    if SWEEP_REUSE:
        cmd.append("--reuse")
    if SWEEP_KEEP_GOING:
        cmd.append("--keep-going")

    run(cmd)
    run_dir = _latest_run_dir(SWEEP_RUNS_DIR, tag)
    strict = _score_submission_csv(out_csv)
    print("\n===", label, "===")
    print("run_dir:", run_dir)
    print("ensemble:", out_csv)
    print("strict score:", strict.get("score"))
    return {"label": label, "tag": tag, "run_dir": str(run_dir), "ensemble_csv": str(out_csv), "strict": strict}


sweep_results = [run_sweep(label, path) for (label, path) in portfolios]

summary_path = OUT_DIR / f"sweep_{SWEEP_SESSION}_summary.json"
summary_path.write_text(json.dumps(sweep_results, indent=2) + "\n", encoding="utf-8")
print("\nWrote:", summary_path)
print("\nDone sweeps:")
for r in sweep_results:
    print("-", r["label"], "score=", r["strict"].get("score"), "run_dir=", r["run_dir"])


+ /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/.venv/bin/python -m santa_packing.cli.score_submission /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/runs/notebook_runs/ensemble_nb_20260106-133853_8b0295c_neigh.csv --nmax 200 --overlap-mode strict --pretty

=== neigh ===
run_dir: /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/runs/notebook_runs/sweep_runs/nb_20260106-133853_8b0295c_neigh_20260106_133854
ensemble: /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/runs/notebook_runs/ensemble_nb_20260106-133853_8b0295c_neigh.csv
strict score: 158.43908215383297
+ /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/.venv/bin/python -m santa_packing.cli.sweep_ensemble --runs-dir /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/runs/notebook_runs/sweep_runs --tag nb_20260106-133853_8b0295c_no_neigh --nmax 200 --seeds 1..20 --jobs 1 --overlap-check selected --recipes-json /home/marcux777/Santa-2025-Christmas-Tree-Packing-Cha

In [9]:
def _read_csv(path: Path) -> list[dict[str, str]]:
    with path.open("r", newline="", encoding="utf-8") as f:
        return list(csv.DictReader(f))


def _print_recipe_breakdown(run_dir: Path) -> None:
    candidates = _read_csv(run_dir / "candidates.csv")
    by_cid = {row["candidate"]: row for row in candidates}
    choices = _read_csv(run_dir / "ensemble_choices.csv")

    # Aggregate: how often each recipe is selected for the final ensemble.
    stats: dict[str, dict[str, float]] = {}
    for row in choices:
        n = int(row["puzzle"])
        cid = row["candidate"]
        s = float(row["s"])
        recipe = by_cid.get(cid, {}).get("recipe", "unknown")
        st = stats.setdefault(recipe, {"count": 0.0, "contrib": 0.0})
        st["count"] += 1.0
        st["contrib"] += (s * s) / float(n)

    total = sum(v["count"] for v in stats.values())
    print("\nRecipe dominance (selection count + score contrib):")
    for recipe, st in sorted(stats.items(), key=lambda kv: (-kv[1]["contrib"], -kv[1]["count"])):
        frac = (st["count"] / total) if total > 0 else 0.0
        print(f"- {recipe:24s}  picked={int(st['count']):3d} ({frac:5.1%})  contrib={st['contrib']:.6f}")


for r in sweep_results:
    rd = Path(r["run_dir"])
    print("\n" + "=" * 80)
    print("Sweep:", r["label"], "score=", r["strict"].get("score"))
    summary = (rd / "summary.md").read_text(encoding="utf-8")
    print(summary)
    _print_recipe_breakdown(rd)

# Quick A/B conclusion (if enabled)
if RUN_AB_NEIGHBORHOOD and len(sweep_results) >= 2:
    by_label = {r["label"]: r for r in sweep_results}
    a = by_label.get("neigh")
    b = by_label.get("no_neigh")
    if a and b:
        sa = float(a["strict"].get("score") or float("nan"))
        sb = float(b["strict"].get("score") or float("nan"))
        print("\nA/B vizinhanca:")
        print("- neigh    :", sa)
        print("- no_neigh :", sb)
        if sa == sa and sb == sb:
            print("- delta (no_neigh - neigh):", sb - sa)



Sweep: neigh score= 158.43908215383297
# Sweep + per-n ensemble

- nmax: 200
- candidates: 80 (recipes=4 seeds=20)
- overlap_check: selected

## Best single candidate
- candidate: mother_refine_neigh_radial_seed15
- score (no overlap check): 175.386865701791
- csv: /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/runs/notebook_runs/sweep_runs/nb_20260106-133853_8b0295c_neigh_20260106_133854/candidates/mother_refine_neigh_radial_seed15/submission.csv

## Ensemble
- score (no overlap check): 158.439082153833
- csv: /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/runs/notebook_runs/sweep_runs/nb_20260106-133853_8b0295c_neigh_20260106_133854/ensemble_submission.csv
- choices: /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/runs/notebook_runs/sweep_runs/nb_20260106-133853_8b0295c_neigh_20260106_133854/ensemble_choices.csv
- candidates table: /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge/runs/notebook_runs/sweep_runs/nb_20260106-133853_8b0295